# The best place to rent

Hill: A person visiting Manhattan can find the best neigbourhood for their needs without needing to do research

Using foursquare and travel time this project will group neighboorhods based on venues in that neighboorhod and venues in neighboorhods that are close in travel time

## Configure the notebook

- Load file with secrets
- Load libraries

The API secrets are not pushed to github. This is handled by placing them in a file named secrets and removing version control from that file.
Since the file is not in the same directory as the notebooks extra code is needed to add the path to the sys search path

In [1]:
import os
secrets_folder_path = os.path.dirname(os.getcwd())
secrets_folder_path
import sys
sys.path.insert(0, secrets_folder_path)

import secrets

The secrets file containts two variables used in the Foursquare API: CLIENT_ID and CLIENT_SECRET
It contains two variables used int the travel time API: APP_ID and API_KEY

In [2]:
# Foursquare 
CLIENT_ID = secrets.CLIENT_ID # your Foursquare ID
CLIENT_SECRET = secrets.CLIENT_SECRET # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30


Import the standard libraries

In [42]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
import json
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print('Libraries imported.')

Libraries imported.


## Get neighborhoods in Manhattan 

The dataset with neighborhoods for New York exists for free on the [web](https://geo.nyu.edu/catalog/nyu_2451_34572)

The information on the webpage where the download link is located gave this information for the element to download a geojson file
```
<a class="btn btn-primary btn-block download download-generated" 
   data-download-path="/download/nyu-2451-34572?type=geojson" 
    data-download="trigger" 
    data-download-type="geojson" 
    data-download-id="nyu-2451-34572" href="">Download
</a>
```

Using this information i ran a request. The information from running a request to the URI: `https://geo.nyu.edu/download/nyu-2451-34572?type=geojson` was:

`[['success',
  '<a data-download="trigger" data-download-id="nyu-2451-34572" data-download-type="generated-geojson" href="/download/file/nyu-2451-34572-geojson.json">Your file nyu-2451-34572-geojson.json is ready for download</a>']]`
  
I used this information to run the second request:

In [59]:
URI_NY = 'https://geo.nyu.edu/download/file/nyu-2451-34572-geojson.json'

response_ny = requests.get(URI_NY)
print(response_ny.status_code, response_ny.reason)

# Get the json
ny_json = response_ny.json()
ny_json

200 OK


{'type': 'FeatureCollection',
 'totalFeatures': 306,
 'features': [{'type': 'Feature',
   'id': 'nyu_2451_34572.1',
   'geometry': {'type': 'Point',
    'coordinates': [-73.84720052054902, 40.89470517661]},
   'geometry_name': 'geom',
   'properties': {'name': 'Wakefield',
    'stacked': 1,
    'annoline1': 'Wakefield',
    'annoline2': None,
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.84720052054902,
     40.89470517661,
     -73.84720052054902,
     40.89470517661]}},
  {'type': 'Feature',
   'id': 'nyu_2451_34572.2',
   'geometry': {'type': 'Point',
    'coordinates': [-73.82993910812398, 40.87429419303012]},
   'geometry_name': 'geom',
   'properties': {'name': 'Co-op City',
    'stacked': 2,
    'annoline1': 'Co-op',
    'annoline2': 'City',
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.82993910812398,
     40.87429419303012,
     -73.82993910812398,
     40.87429419303012]}},
  {'type': 'Feature',
 

Turn the json file into  a data frame

In [65]:
neighborhoods_data = ny_json['features']

# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods_ny = pd.DataFrame(columns=column_names)

for data in neighborhoods_data:
    borough = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods_ny = neighborhoods_ny.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [66]:
neighborhoods_ny.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [68]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods_ny['Borough'].unique()),
        neighborhoods_ny.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


Limit to only Manhattan and take away the borough column

In [139]:
neighborhoods = neighborhoods_ny.loc[neighborhoods_ny['Borough'] == 'Manhattan', 
                     ['Neighborhood', 'Latitude', 'Longitude']]
neighborhoods.reset_index(drop=True)
# REINDEX SO IT WORKS!

,Neighborhood,Latitude,Longitude
0,Marble Hill,40.876551,-73.910660
1,Chinatown,40.715618,-73.994279
2,Washington Heights,40.851903,-73.936900
3,Inwood,40.867684,-73.921210
4,Hamilton Heights,40.823604,-73.949688
5,Manhattanville,40.816934,-73.957385
6,Central Harlem,40.815976,-73.943211
7,East Harlem,40.792249,-73.944182
8,Upper East Side,40.775639,-73.960508
9,Yorkville,40.775930,-73.947118


## Get the foursquare data for Manhattan

Using the foursquare API, get the venues in New York

In [146]:
neighborhood_latitude = neighborhoods.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = neighborhoods.loc[0, 'Longitude'] # neighborhood longitude value
neighborhood_name = neighborhoods.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

KeyError: 'the label [0] is not in the [index]'

## Get the travel time between the different neighborhoods

Using the traveltime API, get the travel time between the different neighborhoods

**Post request example**

```
POST /v4/time-filter HTTP/1.1
Host: api.traveltimeapp.com
Content-Type: application/json
Accept: application/json
X-Application-Id: APP_ID
X-Api-Key: API_KEY
```

Convert the information in the post request example to URI and header

In [30]:
URI_traveltime = 'HTTPS://api.traveltimeapp.com/v4/time-filter'
headers = {'Host': 'api.traveltimeapp.com',
           'Content-Type': 'application/json', 
           'Accept': 'application/json', 
           'X-Application-Id': secrets.APP_ID, # APP ID is in separate file not under version control
           'X-Api-Key': secrets.API_KEY} # API_KEY is in separate file not under version control

**JSON data as raw string**

The json request is long, double clik to view

In [31]:
raw_json_text="""{
  "locations": [
    {
      "id": "London center",
      "coords": {
        "lat": 51.508930,
        "lng": -0.131387
      }
    },
    {
      "id": "Hyde Park",
      "coords": {
        "lat": 51.508824,
        "lng": -0.167093
      }
    },
    {
      "id": "ZSL London Zoo",
      "coords": {
        "lat": 51.536067,
        "lng": -0.153596
      }
    }
  ],
  "departure_searches": [
    {
      "id": "forward search example",
      "departure_location_id": "London center",
      "arrival_location_ids": [
        "Hyde Park",
        "ZSL London Zoo"
      ],
      "transportation": {
        "type": "bus"
      },
      "departure_time": "2019-01-11T08:00:00Z",
      "travel_time": 1800,
      "properties": [
        "travel_time"
      ],
      "range": {
        "enabled": true,
        "max_results": 3,
        "width": 600
      }
    }
  ],
  "arrival_searches": [
    {
      "id": "backward search example",
      "departure_location_ids": [
        "Hyde Park",
        "ZSL London Zoo"
      ],
      "arrival_location_id": "London center",
      "transportation": {
        "type": "public_transport"
      },
      "arrival_time": "2019-01-11T08:00:00Z",
      "travel_time": 1900,
      "properties": [
        "travel_time",
        "distance",
        "distance_breakdown",
        "fares"
      ]
    }
  ]
}"""

Make raw json string into json

In [98]:
cleaned_json_text = (raw_json_text.replace('\n','')
                                .replace(" ", ""))
data = json.loads(cleaned_json_text)
data

{'locations': [{'id': 'Londoncenter',
   'coords': {'lat': 51.50893, 'lng': -0.131387}},
  {'id': 'HydePark', 'coords': {'lat': 51.508824, 'lng': -0.167093}},
  {'id': 'ZSLLondonZoo', 'coords': {'lat': 51.536067, 'lng': -0.153596}}],
 'departure_searches': [{'id': 'forwardsearchexample',
   'departure_location_id': 'Londoncenter',
   'arrival_location_ids': ['HydePark', 'ZSLLondonZoo'],
   'transportation': {'type': 'bus'},
   'departure_time': '2019-01-11T08:00:00Z',
   'travel_time': 1800,
   'properties': ['travel_time'],
   'range': {'enabled': True, 'max_results': 3, 'width': 600}}],
 'arrival_searches': [{'id': 'backwardsearchexample',
   'departure_location_ids': ['HydePark', 'ZSLLondonZoo'],
   'arrival_location_id': 'Londoncenter',
   'transportation': {'type': 'public_transport'},
   'arrival_time': '2019-01-11T08:00:00Z',
   'travel_time': 1900,
   'properties': ['travel_time', 'distance', 'distance_breakdown', 'fares']}]}

Make the post request and check that it is OK

In [33]:
r = requests.post(URI_traveltime, headers=headers, json=data)

print(r.status_code, r.reason)

200 OK


**This is an example of the json data in the request**
Double click to see the example

```
{
  "locations": [
    {
      "id": "London center",
      "coords": {
        "lat": 51.508930,
        "lng": -0.131387
      }
    },
    {
      "id": "Hyde Park",
      "coords": {
        "lat": 51.508824,
        "lng": -0.167093
      }
    },
    {
      "id": "ZSL London Zoo",
      "coords": {
        "lat": 51.536067,
        "lng": -0.153596
      }
    }
  ],
  "departure_searches": [
    {
      "id": "forward search example",
      "departure_location_id": "London center",
      "arrival_location_ids": [
        "Hyde Park",
        "ZSL London Zoo"
      ],
      "transportation": {
        "type": "bus"
      },
      "departure_time": "2019-01-11T08:00:00Z",
      "travel_time": 1800,
      "properties": [
        "travel_time"
      ],
      "range": {
        "enabled": true,
        "max_results": 3,
        "width": 600
      }
    }
  ],
  "arrival_searches": [
    {
      "id": "backward search example",
      "departure_location_ids": [
        "Hyde Park",
        "ZSL London Zoo"
      ],
      "arrival_location_id": "London center",
      "transportation": {
        "type": "public_transport"
      },
      "arrival_time": "2019-01-11T08:00:00Z",
      "travel_time": 1900,
      "properties": [
        "travel_time",
        "distance",
        "distance_breakdown",
        "fares"
      ]
    }
  ]
}
```

In [39]:
r.json()

{'results': [{'search_id': 'backwardsearchexample',
   'locations': [{'id': 'HydePark',
     'properties': [{'travel_time': 1892,
       'distance': 0,
       'distance_breakdown': [{'mode': 'bus', 'distance': 2879},
        {'mode': 'walk', 'distance': 826}],
       'fares': {'breakdown': [{'modes': ['bus'],
          'route_part_ids': [5],
          'tickets': [{'type': 'single', 'price': 1.5, 'currency': 'GBP'}]},
         {'modes': ['bus'],
          'route_part_ids': [6, 5, 7],
          'tickets': [{'type': 'week', 'price': 21, 'currency': 'GBP'},
           {'type': 'month', 'price': 80.7, 'currency': 'GBP'},
           {'type': 'year', 'price': 840, 'currency': 'GBP'}]},
         {'modes': ['bus'],
          'route_part_ids': [7],
          'tickets': [{'type': 'single', 'price': 1.5, 'currency': 'GBP'}]},
         {'modes': ['bus'],
          'route_part_ids': [6],
          'tickets': [{'type': 'single', 'price': 1.5, 'currency': 'GBP'}]}],
        'tickets_total': [{'type': 

## Visualize the map 

Use the geocoding from travel time to get longetiude and altitude of manhattan

Use the longitude and latitude to make a map of manhattan

Add labels for every neigbouthoods into the manhattan map

In [113]:
query = 'Central Park, New York'
URI_traveltime = 'HTTPS://api.traveltimeapp.com/v4/geocoding/search?query={}'.format(query)
print('URI is : {}'.format(URI_traveltime))
headers = {'Host': 'api.traveltimeapp.com',
           'Accept': 'application/json', 
           'X-Application-Id': secrets.APP_ID, # APP ID is in separate file not under version control
           'X-Api-Key': secrets.API_KEY} # API_KEY is in separate file not under version control

URI is : HTTPS://api.traveltimeapp.com/v4/geocoding/search?query=Central Park, New York


In [114]:
r = requests.get(URI_traveltime, headers=headers)

print(r.status_code, r.reason)
r.url

200 OK


'https://api.traveltimeapp.com/v4/geocoding/search?query=Central%20Park,%20New%20York'

In [115]:
r.json()

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'geometry': {'type': 'Point', 'coordinates': [-73.965575, 40.782476]},
   'properties': {'name': 'Central Park',
    'label': 'Central Park, New York County, NY, USA',
    'score': 0.6,
    'region': 'New York',
    'region_code': 'NY',
    'neighbourhood': 'Central Park',
    'county': 'New York County',
    'country': 'United States',
    'country_code': 'USA',
    'continent': 'North America',
    'features': {'public_transport': {'date_start': '2018-11-21T06:00:00+01:00',
      'date_end': '2019-01-24T06:00:00+01:00'},
     'fares': False,
     'postcodes': False}}},
  {'type': 'Feature',
   'geometry': {'type': 'Point', 'coordinates': [-78.841555, 42.944712]},
   'properties': {'name': 'Central Park',
    'label': 'Central Park, Buffalo, NY, USA',
    'score': 0.6,
    'region': 'New York',
    'region_code': 'NY',
    'neighbourhood': 'Central Park',
    'county': 'Erie County',
    'city': 'Buffalo',
    'country

In [126]:
manhattan_location_data = r.json()['features'][0]
[longitude, latitude] = manhattan_location_data['geometry']['coordinates']

In [131]:
# create map of Manhattan using latitude and longitude values
map_manhattan = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, label in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_manhattan)  
map_manhattan